# dMRI Preprocessing

This note book processes multiband diffusion weighted imaging for tensor-based analyses

In [1]:
from nipype.pipeline.engine import Workflow, Node, JoinNode, MapNode
from nipype.interfaces.utility import IdentityInterface, Function
from nipype.interfaces.io import SelectFiles, DataSink, DataGrabber, FreeSurferSource
from nipype.interfaces.fsl import BET, MeanImage, ApplyTOPUP, TOPUP, Merge,ExtractROI
from nipype.interfaces.ants import RegistrationSynQuick, ApplyTransforms
from nipype.interfaces.freesurfer import FSCommand, MRIConvert

# MATLAB setup - Specify path to current SPM and the MATLAB's default mode
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('~/spm12/toolbox')
MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")

# FSL set up- change default file output type
from nipype.interfaces.fsl import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

# Study-specific variables
project_home = '/data/perlman/moochie/user_data/CamachoCat/ChEC/dmri_proc'
output_dir = project_home + '/proc/preprocessing'
workflow_dir = project_home + '/workflows'
raw_dir = '/data/perlman/moochie/study_data/ChEC/MRI_data'
phase_encoding_file = project_home + '/misc/chec_encoding_file.txt'

template_2mm = '/data/perlman/moochie/user_data/CamachoCat/Aggregate_anats/templates/lcbd_template_2mm_brain.nii.gz'
subjects_list = open(project_home + '/misc/chec_subjects.txt').read().splitlines()

200511-12:58:23,97 nipype.utils INFO:
	 No new version available.


In [2]:
### Universal nodes

# get subjects list
infosource = Node(IdentityInterface(fields=['subjid']),
                  name='infosource')
infosource.iterables = [('subjid', subjects_list)]

# Sink data of interest
substitutions = [('_subjid_', '')] #output file name substitutions
datasink = Node(DataSink(base_directory = output_dir,
                        container = output_dir,
                        substitutions = substitutions), 
                name='datasink')

## Unwarping workflow

This workflow includes both unwarping and eddy correction.

In [3]:
# select dMRI data
dmri_template = {'dmri_pe0': raw_dir + '/sub-{subjid}/dwi/sub-{subjid}_98dir_AP.nii.gz',
                 'bval':raw_dir + '/sub-{subjid}/dwi/sub-{subjid}_98dir_AP.bval',
                 'bvec':raw_dir + '/sub-{subjid}/dwi/sub-{subjid}_98dir_AP.bvec'}
selectdmri = Node(SelectFiles(dmri_template), name='selectdmri')

pes_template={'pes':raw_dir + '/sub-{subjid}/dwi/sub-{subjid}_98dir_{pe}.nii.gz'}
selectpes = Node(SelectFiles(pes_template),name='selectpes')
selectpes.iterables=('pe',['AP','PA'])

# include only the first volume of each PE volume
trim_pes = Node(ExtractROI(t_min=0,t_size=1,roi_file='pe_trimmed.nii.gz'),name='trim_pes')

# merge to 1 file for topup to calculate the fieldcoef
merge_pes = JoinNode(Merge(dimension='t',
                           merged_file='merged_pes.nii.gz'),
                     name='merge_pes', joinsource='selectpes', joinfield='in_files')

# actually do the unwarping
topup = Node(TOPUP(encoding_file=phase_encoding_file), name='topup')

apply_topup = Node(ApplyTOPUP(in_index=[1], encoding_file=phase_encoding_file,
                              method='jac', out_corrected='dmri_unwarped.nii.gz'),
                   name='apply_topup')

# average dMRI so we can make a mask
avg_dmri = Node(MeanImage(), name='avg_dmri')

# create a dMRI space brain mask using BET
make_mask = Node(BET(mask=True), name='make_mask')

In [ ]:
prepreprocflow = Workflow(name='prepreprocflow')
prepreprocflow.connect([(infosource,selectdmri, [('subjid','subjid')]),
                        (infosource, selectpes, [('subjid','subjid')]),
                        (selectpes, trim_pes, [('pes','in_file')]),
                        (trim_pes, merge_pes,[('roi_file','in_files')]),
                        (merge_pes, topup, [('merged_file','in_file')]),
                        (topup, apply_topup, [('out_fieldcoef','in_topup_fieldcoef'), 
                                              ('out_movpar','in_topup_movpar')]),
                        (selectdmri, apply_topup, [('dmri_pe0','in_files')]),
                        (apply_topup, avg_dmri, [('out_corrected','in_file')]),
                        (avg_dmri, make_mask, [('out_file','in_file')]),
                        (make_mask, datasink, [('mask_file','in_mask')]),
                        (apply_topup, datasink, [('out_corrected','unwarped_niftis')])
                       ])

prepreprocflow.base_dir = workflow_dir
#prepreprocflow.write_graph(graph2use='flat')
prepreprocflow.run('MultiProc', plugin_args={'n_procs': 4, 'memory_gb':10})

## Preprocessing for subjects without both encoding directions
This workflow makes group a average fieldmap as done in Laumann et al. 2015

In [4]:
def make3DTemplate(subject_vols, num_proc, output_prefix):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    
    from os.path import abspath, split
    from os import getcwd
    from shutil import copyfile
    import subprocess 

    curr_dir = getcwd()

    #copy data into current directory
    for T in range(0,len(subject_vols)):
        [dirname,filename] = split(subject_vols[T])
        copyfile(subject_vols[T],curr_dir + '/' + str(T)+'_'+filename)

    # -c flag is control for local computing (2= use localhost; required for -j flag)
    # -j flag is for number of processors allowed

    subprocess.check_call(['antsMultivariateTemplateConstruction2.sh','-d','3','-o', 
                           output_prefix,'-r','1','-c','2','-n','0','-j', str(num_proc), '*.nii.gz'], shell=True)
    
    sample_template = abspath(output_prefix + 'template0.nii.gz')
    
    return(sample_template)

In [16]:
# sort IDs with good and bad fieldmaps
subs_corr_fmap = ['1000', '1001', '1008', '1011', '1016', '1017', '1027', '1031',
                  '1034', '1036', '1040', '1045', '1047', '1048', '1052', '1053'] #used to make group mean fieldmap
subs_to_unwarp = ['1006', '1007', '1012', '1015', '1023', '1025', '1026', '1032',
                  '1046', '1055']
# get subjects list
infosource = Node(IdentityInterface(fields=['subjid']),
                  name='infosource')
infosource.iterables = [('subjid', subs_to_unwarp)]

# select good pes
pe_template={'pe': project_home + '/misc/%s_pe_template0.nii.gz'}
grab_pe_data = Node(DataGrabber(field_template=pe_template, 
                                sort_filelist=True,
                                base_directory=raw_dir, 
                                template=project_home + '/misc/%s_pe_template0.nii.gz', 
                                infields=['dir'], 
                                template_args={'pe':[['dir']]}), 
                    name='grab_pe_data')
grab_pe_data.iterables=('dir',['AP','PA'])

# grab just first B0 volume for registration
trim_b0 = Node(ExtractROI(t_min=0, t_size=1), name='trim_b0')

# merge to 1 file for topup to calculate the fieldcoef
merge_pes = JoinNode(Merge(dimension='t',
                           merged_file='merged_pes.nii.gz'),
                     name='merge_pes', joinsource='grab_pe_data', joinfield='in_files')

#register template to subject data
register2subject = Node(RegistrationSynQuick(dimension=3,
                                             transform_type='a'), name='register2subject')

In [ ]:
groupunwarp = Workflow(name='groupunwarp')
groupunwarp.connect([(infosource,selectdmri, [('subjid','subjid')]),
                     (grab_pe_data, register2subject,[('pe','moving_image')]),
                     (selectdmri, trim_b0, [('dmri_pe0','in_file')]),
                     (trim_b0, register2subject, [('roi_file','fixed_image')]),
                     (register2subject, merge_pes, [('warped_image','in_files')]),
                     (merge_pes, topup, [('merged_file','in_file')]),
                     (topup, apply_topup, [('out_fieldcoef','in_topup_fieldcoef'), 
                                           ('out_movpar','in_topup_movpar')]),
                     (selectdmri, apply_topup, [('dmri_pe0','in_files')]),
                     (apply_topup, avg_dmri, [('out_corrected','in_file')]),
                     (avg_dmri, make_mask, [('out_file','in_file')]),
                     (make_mask, datasink, [('mask_file','in_mask')]),
                     (apply_topup, datasink, [('out_corrected','unwarped_niftis')])
                    ])

groupunwarp.base_dir = workflow_dir
#groupunwarp.write_graph(graph2use='flat')
groupunwarp.run('MultiProc', plugin_args={'n_procs': 4, 'memory_gb':10})

200512-13:32:22,681 nipype.workflow INFO:
	 Workflow groupunwarp settings: ['check', 'execution', 'logging', 'monitoring']
200512-13:32:22,751 nipype.workflow INFO:
	 Running in parallel.
200512-13:32:22,755 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 12 jobs ready. Free memory (GB): 10.00/10.00, Free processors: 4/4.
200512-13:32:22,817 nipype.workflow INFO:
	 [Node] Setting-up "groupunwarp.grab_pe_data" in "/data/perlman/moochie/user_data/CamachoCat/ChEC/dmri_proc/workflows/groupunwarp/_dir_PA/grab_pe_data".
200512-13:32:22,820 nipype.workflow INFO:
	 [Node] Setting-up "groupunwarp.selectdmri" in "/data/perlman/moochie/user_data/CamachoCat/ChEC/dmri_proc/workflows/groupunwarp/_subjid_1055/selectdmri".
200512-13:32:22,820 nipype.workflow INFO:
	 [Node] Setting-up "groupunwarp.grab_pe_data" in "/data/perlman/moochie/user_data/CamachoCat/ChEC/dmri_proc/workflows/groupunwarp/_dir_AP/grab_pe_data".
200512-13:32:22,821 nipype.workflow INFO:
	 [Node] Setting-up "groupunwarp.sel